In [47]:
import os

if "src" not in os.listdir():
    os.chdir("../../../")

import numpy as np
from dateutil import parser
import json
import pickle
from keras.models import load_model
from datetime import timezone
import pandas as pd

from src.lib.stock_dataset import read_stock_dataset
from src.lib.activations import negative_softmax

In [48]:
path = "results/1_lstm_ptrade/"
keys = "alpaca_config.json"

model = load_model(
    os.path.join(path, "model.h5"),
    custom_objects={
        "negative_softmax": negative_softmax
    }
)

with open(os.path.join(path, "config.json"), "r") as f:
    config = json.load(f)

n_time_steps = config["n_time_steps"]
symbols = config["symbols"]
target_column = config["target_column"]

with open(os.path.join(path, "scaler.pkl"), "rb") as f:
    scaler = pickle.load(f)

ta_preprocess_candles = 296

In [49]:
dataset = read_stock_dataset(symbols, target_column, n_time_steps, transform=True)

In [50]:
dataset = dataset.apply_standard_scaler(scaler)

In [51]:
# path = "results/1_lstm_ptrade/Xs"
# files = os.listdir(path)
# files.sort()
# print(files)
# file_name = files[5]
# file_path = os.path.join(path, file_name)
# X = np.load(file_path)
# print(X)

In [52]:
path = "results/1_lstm_ptrade/Xs"
files = os.listdir(path)
print(files)
for file_name in files:
    time = parser.parse(file_name[:-len(".npy")])
    time = time.replace(second=0, microsecond=0)
    print(time)
    if not time in dataset.X.index:
        print("continueing")
        continue

    file_path = os.path.join(path, file_name)
    X = np.load(file_path)
    print((X==np.nan).sum())
    # X[0] = scaler.transform(X[0])
    i = dataset.X.index.get_loc(time)
    df_chunk = dataset.X.iloc[i-n_time_steps: i]
    # print(time)
    # print(df_chunk)
    X_hist = df_chunk.to_numpy()[np.newaxis, :, :]

    # assert (X == X_hist).all()
    m = 6
    diff = np.abs(X_hist - X)[0]
    diff = diff.mean(0)
    print(diff.shape)
    # diff = diff[0, 8, :]
    # print(diff)
    # print(X_hist[0, :, 8])
    for symbol, value in zip(dataset.X.columns, diff):
        print(symbol, value)
    # print((X_hist[0, :, m] - X[0, :, m]).sum())

['2021-09-21 15:48:03.317500+00:00.npy', '2021-09-21 15:52:03.199651+00:00.npy', '2021-09-21 15:49:03.160159+00:00.npy', '2021-09-21 15:51:03.274808+00:00.npy', '2021-09-21 15:46:03.579967+00:00.npy', '2021-09-21 15:53:03.293813+00:00.npy', '2021-09-21 15:50:03.308053+00:00.npy', '2021-09-21 15:47:03.192641+00:00.npy']
2021-09-21 15:48:00+00:00
0
(26,)
SPY_volume 0.0
SPY_trades 0.0
SPY_oc_ret 0.0
SPY_pdiff_open 0.0
SPY_pdiff_high 0.0
SPY_pdiff_low 0.0
SPY_pdiff_vwap 0.0
SPY_ma_50 4.394596665735495e-14
SPY_ma_200 1.789714210165272e-14
SPY_rsi 0.0
SPY_macd 0.0
SPY_bbands_b 3.744915682340143e-09
SPY_bbands_p 5.913050425478974e-09
SPXS_volume 0.0
SPXS_trades 0.0
SPXS_oc_ret 0.0
SPXS_pdiff_open 0.0
SPXS_pdiff_high 0.0
SPXS_pdiff_low 0.0
SPXS_pdiff_vwap 0.0
SPXS_ma_50 9.64939933512099e-15
SPXS_ma_200 6.097119337189483e-15
SPXS_rsi 0.0
SPXS_macd 0.0
SPXS_bbands_b 4.392666755924162e-09
SPXS_bbands_p 2.184720515390967e-08
2021-09-21 15:52:00+00:00
0
(26,)
SPY_volume 5.092948540154352e-08
SPY_tr